#  Relax Inc. Take-Home Challenge 
## August Perez

Springboard Data Science track unit 37.2

In [1]:
#imports

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import datetime as dt
from scipy.stats import chi2_contingency


import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose


from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler


#
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
    #reminder code so I can ignore warnings for single code cells:
    # warnings.filterwarnings('ignore')
    # warnings.resetwarnings()

In [2]:
# Load the data
df_user = pd.read_csv('takehome_users.csv', encoding='ISO-8859-1')
df_engage = pd.read_csv('takehome_user_engagement.csv')

In [3]:
print("User Data:")
df_user.head()

User Data:


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [5]:
df_user.describe(include='all')

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
count,12000.00000,12000,12000,12000,12000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000
unique,NaN,11996,11355,11980,5,NaN,NaN,NaN,NaN,NaN
top,NaN,2014-02-11 17:57:53,Araujo Gabriela,AlfieLane@yahoo.com,ORG_INVITE,NaN,NaN,NaN,NaN,NaN
freq,NaN,2,5,2,4254,NaN,NaN,NaN,NaN,NaN
mean,6000.50000,NaN,NaN,NaN,NaN,1.379279e+09,0.249500,0.149333,141.884583,5962.957145
std,3464.24595,NaN,NaN,NaN,NaN,1.953116e+07,0.432742,0.356432,124.056723,3383.761968
min,1.00000,NaN,NaN,NaN,NaN,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,NaN,NaN,NaN,NaN,1.363195e+09,0.000000,0.000000,29.000000,3058.000000
50%,6000.50000,NaN,NaN,NaN,NaN,1.382888e+09,0.000000,0.000000,108.000000,5954.000000
75%,9000.25000,NaN,NaN,NaN,NaN,1.398443e+09,0.000000,0.000000,238.250000,8817.000000


___

In [6]:
print("\nEngagement Data:")
df_engage.head()


Engagement Data:


,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [7]:
df_engage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [8]:
df_engage.describe(include='all')

,time_stamp,user_id,visited
count,207917,207917.000000,207917.0
unique,207220,NaN,NaN
top,2013-04-06 21:21:37,NaN,NaN
freq,2,NaN,NaN
mean,NaN,5913.314197,1.0
std,NaN,3394.941674,0.0
min,NaN,1.000000,1.0
25%,NaN,3087.000000,1.0
50%,NaN,5682.000000,1.0
75%,NaN,8944.000000,1.0


___
Change col names

edits for clarity, consistency, & succinctness (helps with reducing plot axis label overlap)

In [9]:
df_user.rename(columns={'object_id' : 'user_id', 'last_session_creation_time' : 'last_login', 'opted_in_to_mailing_list' : 'mailing_list', 'enabled_for_marketing_drip' : 'marketing_drip'}, inplace=True)
df_user.head(2)

,user_id,creation_time,name,email,creation_source,last_login,mailing_list,marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0


___

# Missing Data

In [10]:
print("\nMissing Values in User Data:")
print(df_user.isnull().sum())
print("\nMissing Values in Engagement Data:")
print(df_engage.isnull().sum())


Missing Values in User Data:
user_id                  0
creation_time            0
name                     0
email                    0
creation_source          0
last_login            3177
mailing_list             0
marketing_drip           0
org_id                   0
invited_by_user_id    5583
dtype: int64

Missing Values in Engagement Data:
time_stamp    0
user_id       0
visited       0
dtype: int64


In [11]:
missing_last_session_rows = df_user[df_user['last_login'].isnull()]

missing_last_session_rows

,user_id,creation_time,name,email,creation_source,last_login,mailing_list,marketing_drip,org_id,invited_by_user_id
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN
11,12,2014-04-17 23:48:38,Mathiesen Lærke,LaerkeLMathiesen@cuvox.de,ORG_INVITE,NaN,0,0,130,9270.0
14,15,2013-07-16 21:33:54,Theiss Ralf,RalfTheiss@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,175,NaN
15,16,2013-02-11 10:09:50,Engel René,ReneEngel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,211,NaN
...,...,...,...,...,...,...,...,...,...,...
11975,11976,2013-12-25 22:01:41,Kohl Leah,LeahKohl@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,248,NaN
11977,11978,2014-04-23 16:28:06,Castro Pedro,PedroCunhaCastro@gustr.com,PERSONAL_PROJECTS,NaN,1,0,29,NaN
11984,11985,2013-07-08 17:23:26,Jespersen Marcus,MarcusTJespersen@cuvox.de,PERSONAL_PROJECTS,NaN,0,0,74,NaN
11992,11993,2013-03-28 23:24:21,Townsend Isabel,IsabelTownsend@cuvox.de,PERSONAL_PROJECTS,NaN,0,0,281,NaN


In [12]:
missing_invited_by_user_rows = df_user[df_user['invited_by_user_id'].isnull()]

missing_invited_by_user_rows

,user_id,creation_time,name,email,creation_source,last_login,mailing_list,marketing_drip,org_id,invited_by_user_id
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN
10,11,2013-12-26 03:55:54,Paulsen Malthe,MaltheAPaulsen@gustr.com,SIGNUP,1.388117e+09,0,0,69,NaN
13,14,2012-10-11 16:14:33,Rivera Bret,BretKRivera@gmail.com,SIGNUP,1.350058e+09,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
11992,11993,2013-03-28 23:24:21,Townsend Isabel,IsabelTownsend@cuvox.de,PERSONAL_PROJECTS,NaN,0,0,281,NaN
11994,11995,2013-10-21 13:33:17,Werfel Sebastian,SebastianWerfel@gmail.com,PERSONAL_PROJECTS,1.382449e+09,0,0,0,NaN
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,1.338638e+09,0,0,6,NaN


Look into df_engage to see if last_login for df_user can be filled in

In [17]:
df_engage.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [28]:
#Looking at top 10 users by count of total logins
    #doesn't exactly apply to problem}
df_engage[['user_id', 'visited']].groupby(by='user_id').sum().sort_values(by='visited', ascending=False).head(10)

,visited
user_id,
3623,606
906,600
1811,593
7590,590
8068,585
9402,581
4022,580
3269,566
9312,547


In [37]:
max_login = df_engage[['user_id', 'time_stamp']].groupby(by='user_id').max()

print('count of users who have ever logged in:\n', len(max_login))
print('count of total users:\n', len(df_user))
print('count of users who have NEVER logged in:\n', len(df_user) - len(max_login))

max_login.head()

count of users who have ever logged in:
 8823
count of total users:
 12000
count of users who have NEVER logged in:
 3177


,time_stamp
user_id,
1,2014-04-22 03:53:30
2,2014-03-31 03:45:04
3,2013-03-19 23:14:52
4,2013-05-22 08:09:28
5,2013-01-22 10:14:20


Users that have never logged in match null count for 'last_login' in df_user.

Plan on imputing pd.NaT to indicate no login (& work nicer when converting to datetime)

___
impute pd.NaT for 'last_login' col

This still is a null but on purpose

In [38]:
df_user['last_login'] = df_user['last_login'].fillna(pd.NaT)

In [46]:
df_user['last_login'].value_counts(dropna=False).head()

last_login
NaT             3177
1401868018.0       2
1401916897.0       2
1398138810.0       1
1381811721.0       1
Name: count, dtype: int64

___
impute invited_by_user_id with 0 to indicate no invitation

In [47]:
df_user['invited_by_user_id'] = df_user['invited_by_user_id'].fillna(0)

In [48]:
df_user['invited_by_user_id'].value_counts(dropna=False).head()

invited_by_user_id
0.0        5583
10741.0      13
2527.0       12
1525.0       11
2308.0       11
Name: count, dtype: int64

In [49]:
df_user.isnull().sum()

user_id                  0
creation_time            0
name                     0
email                    0
creation_source          0
last_login            3177
mailing_list             0
marketing_drip           0
org_id                   0
invited_by_user_id       0
dtype: int64